In [1]:
import logging
from naslib.defaults.trainer import Trainer
from naslib.optimizers import DARTSOptimizer
from naslib.search_spaces import DartsSearchSpace
from naslib.utils import utils, setup_logger, get_config_from_args, set_seed, log_args
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from fvcore.common.config import CfgNode
from copy import deepcopy
from IPython.display import clear_output
import torch

device: cuda:0
device: cpu
device: cuda:0
device: cuda:0
device: cuda:0
device: cuda:0


In [3]:
config = utils.get_config_from_args(config_type='nas')
config.optimizer = 'darts'
utils.set_seed(config.seed)
clear_output(wait=True)
utils.log_args(config)

logger = setup_logger(config.save + '/log.log')
logger.setLevel(logging.INFO)

[06/29 15:00:21 nl.utils.utils]: dataset....................................cifar10
[06/29 15:00:21 nl.utils.utils]: seed.............................................0
[06/29 15:00:21 nl.utils.utils]: search_space...........................nasbench201
[06/29 15:00:21 nl.utils.utils]: out_dir........................................run
[06/29 15:00:21 nl.utils.utils]: optimizer....................................darts
[06/29 15:00:21 nl.utils.utils]: searchacq_fn_optimization: random_sampling
acq_fn_type: its
arch_learning_rate: 0.0003
arch_weight_decay: 0.001
batch_size: 256
checkpoint_freq: 1000
cutout: False
cutout_length: 16
cutout_prob: 1.0
data_size: 25000
debug_predictor: False
drop_path_prob: 0.0
encoding_type: adjacency_one_hot
epochs: 100
fidelity: -1
gpu: None
grad_clip: 5
k: 10
learning_rate: 0.025
learning_rate_min: 0.001
max_mutations: 1
momentum: 0.9
num_arches_to_mutate: 2
num_candidates: 20
num_ensemble: 3
num_init: 10
output_weights: True
population_size: 30
predictor_t

In [4]:
class Minimum(ops.AbstractPrimitive):

    def __init__(self, **kwargs):
        super().__init__(locals())

    def forward(self, x_1, x_2, edge_data):
        return torch.minimum(x_1, x_2)

    def get_embedded_ops(self):
        return None
    

class Maximum(ops.AbstractPrimitive):

    def __init__(self, **kwargs):
        super().__init__(locals())

    def forward(self, x_1, x_2, edge_data):
        return torch.maximum(x_1, x_2)

    def get_embedded_ops(self):
        return None

In [5]:
class SimpleSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'a_stage_2'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        stages = ['a_stage_1', 'a_stage_2']

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2) # intermediate node
        activation_cell.add_node(3) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData())]) # mutable intermediate edge
        activation_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable output edge

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        for i, scope in zip(range(3, 5), stages):
            self.add_node(i, subgraph=deepcopy(activation_cell).set_scope(scope).set_input([i-1])) # activation cell i
            self.nodes[i]['subgraph'].name = scope
        self.add_node(5) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 5)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[4, 5].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in stages:
            self.update_edges(
                update_func=lambda edge: self._set_ops(edge),
                scope=scope,
                private_edge_data=True,
            )

    def _set_ops(self, edge):
        edge.data.set('op', [
            ops.Sequential(nn.ReLU()),
            ops.Sequential(nn.Hardswish()),
            ops.Sequential(nn.LeakyReLU()),
            ops.Sequential(nn.Identity())
        ])

In [19]:
class ComplexSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'u_stage_1',
        'u_stage_2',
        'b_stage_1'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        u_stages = ['u_stage_1', 'u_stage_2']
        
        # unary cell definition
        unary_cell = Graph()
        unary_cell.name = 'u_cell'
        unary_cell.add_node(1) # input node
        unary_cell.add_node(2) # intermediate node
        unary_cell.add_node(3) # output node
        unary_cell.add_edges_from([(1, 2, EdgeData())]) # mutable edge
        unary_cell.edges[1, 2].set('cell_name', 'u_cell')
        unary_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable edge
        
        # binary cell definition
        binary_cell = Graph()
        binary_cell.name = 'b_cell'
        binary_cell.add_node(1) # input node
        binary_cell.add_node(2) # input node
        binary_cell.add_node(3) # concatination node
        binary_cell.nodes[3]['comb_op'] = lambda tensors : tensors
        binary_cell.add_node(4) # intermediate node
        binary_cell.add_node(5) # output node
        binary_cell.add_edges_from([(3, 4, EdgeData())]) # mutable edge
        binary_cell.edges[3, 4].set('cell_name', 'b_cell') 
        binary_cell.add_edges_from([(1, 3, EdgeData().finalize()),
                                    (2, 3, EdgeData().finalize()),
                                    (4, 5, EdgeData().finalize())]) # immutable edges
        
        # activation cell definition
        activation_cell = Graph()
        activation_cell.name = 'a_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2, subgraph=deepcopy(unary_cell).set_scope('u_stage_1').set_input([1])) # unary node
        activation_cell.nodes[2]['subgraph'].name = 'u_stage_1'
        activation_cell.add_node(3, subgraph=deepcopy(unary_cell).set_scope('u_stage_2').set_input([1])) # unary node
        activation_cell.nodes[3]['subgraph'].name = 'u_stage_2'
        activation_cell.add_node(4, subgraph=deepcopy(binary_cell).set_scope('b_stage_1').set_input([2, 3])) # binary node
        activation_cell.nodes[4]['subgraph'].name = 'b_stage_1'
        activation_cell.add_node(5) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData().finalize()), 
                                        (1, 3, EdgeData().finalize()),
                                        (2, 4, EdgeData().finalize()),
                                        (3, 4, EdgeData().finalize()), 
                                        (4, 5, EdgeData().finalize())])
        
        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3, subgraph=deepcopy(activation_cell).set_input([2])) # activation cell
        self.nodes[3]['subgraph'].name = 'a_stage_1'
        self.add_node(4) # output node
        self.add_edges_from([(i, i+1) for i in range(1, 4)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[3, 4].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in u_stages:
            self.update_edges(
                update_func=lambda edge: self._set_unary_ops(edge),
                scope=scope,
                private_edge_data=True,
            ) # set unary cell ops
        
        self.update_edges(
            update_func=lambda edge: self._set_binary_ops(edge),
            scope='b_stage_1',
            private_edge_data=True
        ) # set binary cell ops
        

    def _set_unary_ops(self, edge):
        edge.data.set('op', [
            ops.Sequential(ops.Identity()),
            ops.Sequential(ops.Zero(stride=1))
        ]) 
        
    def _set_binary_ops(self, edge):
        edge.data.set('op', [
            ops.Sequential(Minimum()),
            ops.Sequential(Maximum())
        ]) 

In [20]:
search_space = ComplexSearchSpace()

In [21]:
optimizer = DARTSOptimizer(config)
optimizer.adapt_search_space(search_space)

[06/29 15:13:37 nl.optimizers.oneshot.darts.optimizer]: Parsed graph:
Graph a_stage_1:
 Graph(
  (a_stage_1-edge(1,2)): Identity()
  (a_stage_1-edge(1,3)): Identity()
  (a_stage_1-subgraph_at(2)): Graph u_stage_1-0.9097463, scope u_stage_1, 3 nodes
  (a_stage_1-edge(2,4)): Identity()
  (a_stage_1-subgraph_at(3)): Graph u_stage_2-0.9097463, scope u_stage_2, 3 nodes
  (a_stage_1-edge(3,4)): Identity()
  (a_stage_1-subgraph_at(4)): Graph b_stage_1-0.9827855, scope b_stage_1, 5 nodes
  (a_stage_1-edge(4,5)): Identity()
)
Graph b_stage_1:
 Graph(
  (b_stage_1-edge(1,3)): Identity()
  (b_stage_1-edge(2,3)): Identity()
  (b_stage_1-edge(3,4)): MixedOp(
    (primitive-0): Sequential(
      (op): Sequential(
        (0): Minimum()
      )
    )
    (primitive-1): Sequential(
      (op): Sequential(
        (0): Maximum()
      )
    )
  )
  (b_stage_1-edge(4,5)): Identity()
)
Graph u_stage_1:
 Graph(
  (u_stage_1-edge(1,2)): MixedOp(
    (primitive-0): Sequential(
      (op): Sequential(
      

In [22]:
trainer = Trainer(optimizer, config)
trainer.search()

[06/29 15:13:44 nl.defaults.trainer]: param size = 0.006882MB
[06/29 15:13:44 nl.defaults.trainer]: Start training
Files already downloaded and verified
Files already downloaded and verified
[06/29 15:13:45 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.000181, -0.000723, 0
+0.000181, -0.001192, 0
+0.000235, -0.000816, 0


TypeError: forward() missing 1 required positional argument: 'edge_data'

In [10]:
trainer.evaluate_oneshot()

[06/28 14:08:23 nl.defaults.trainer]: Start one-shot evaluation
Files already downloaded and verified
Files already downloaded and verified
[06/28 14:08:29 nl.defaults.trainer]: Evaluation finished


58.43333332722982